In [1]:
import json
from openai import OpenAI
from pydantic import BaseModel

In [2]:
# Options
class Opt:
    apikey = "sk-a0e74b59372f4cf7ab4b6bfd5d3c718d"
    baseurl = "https://api.deepseek.com/v1"
    dataset_path:str = "data/sharegpt_zh_38K_format.jsonl"
    dataset:list = []

In [3]:
class Msg(BaseModel):
    role: str
    content: str

client = OpenAI(api_key=Opt.apikey, base_url=Opt.baseurl)
def chat(client:OpenAI, question:str, history:list[Msg]=[], verbose:bool=True, temperature=0.0) -> tuple[str, list]:
    history.append({"role":"user", "content":question})
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=history,
        stream=False,
        temperature=temperature
    )
    resp = response.choices[0].message.content
    if verbose: print(resp)
    history.append({"role":"assistant", "content":resp})
    return resp, history


In [4]:
quest = "python sqlite3 中的fetchmany怎么用？"
ori_hist = [{"role":"system", "content":"请根据我提出的问题扮演相应领域的专业人士回答问题，请尽可能详细，全面地回答。可以借助实际例子或生动的比喻说明回答中较为晦涩的部分。如果我的问题描述不够详细、过于宽泛，请猜测我的接下来要问什么或我的意图是什么并提前回答。"}]
resp, _ = chat(client, quest, ori_hist)

In [ ]:
squeeze_prompt = f"""阅读以下资料并完成我提出的任务：
{resp}

【任务】阅读以上资料，用一小段话简洁地回答"{quest}"。如果需要总结的内容有多个并列条目，请抽取最重要的一点进行总结，其他情况直接带过。如果一定需要总结代码（即没有代码就完全不能回答问题的情况），代码的实现思路即可，不要在回答中给出任何纯代码。"""
print(squeeze_prompt)

阅读以下资料并完成我提出的任务：
在Python中使用`sqlite3`模块时，`fetchmany`方法用于从查询结果中获取指定数量的记录。这个方法特别适用于处理大量数据时，避免一次性将所有数据加载到内存中，从而提高性能。

下面是一个详细的示例，展示如何使用`fetchmany`方法：

```python
import sqlite3

# 连接到SQLite数据库
# 如果数据库不存在，会自动在当前目录创建一个数据库文件
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 创建一个示例表
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER
)
''')

# 插入一些示例数据
cursor.execute("INSERT INTO users (name, age) VALUES ('Alice', 30)")
cursor.execute("INSERT INTO users (name, age) VALUES ('Bob', 25)")
cursor.execute("INSERT INTO users (name, age) VALUES ('Charlie', 35)")
cursor.execute("INSERT INTO users (name, age) VALUES ('David', 28)")
cursor.execute("INSERT INTO users (name, age) VALUES ('Eve', 22)")

# 提交事务
conn.commit()

# 查询数据并使用fetchmany获取指定数量的记录
cursor.execute("SELECT * FROM users")

# 使用fetchmany获取3条记录
rows = cursor.fetchmany(3)
print("First 3 rows:")
for row in rows:
    print(row)

# 再次使用fetchmany获取接下来的2条记录
rows = cursor.fetc

In [ ]:
new_resp, _ = chat(client, squeeze_prompt, temperature=0.7)

在Python的`sqlite3`模块中，`fetchmany`方法用于从查询结果中分批获取指定数量的记录，避免一次性加载所有数据到内存中，从而提高性能。通过指定参数，可以控制每次获取的记录数量，并且可以多次调用该方法以逐步获取所有记录。


In [ ]:
addtone_prompt = f"""假设这是两个可爱的女孩子正在网上交流。女孩们都喜欢的话题是动漫、科技、宠物、甜点，是二次元宅女。她们对编程、电子、自然科学、法律金融等专业有深入了解，她们都喜欢使用颜文字。
请用更加符合人物设定的语气风格改写这段对话，不改变原文意思，不改变专业词汇。需要使用缩略语，网络用语，简称代称，省略主语定语等方法达成简洁的网络对话效果。可以加入语气词和从句，使用同音不同字的语气后缀。：

A：{quest}
B：{new_resp}

按照json格式输出改写后的对话。示例输出："""+\
"""
{
    "A":"abcd",
    "B":"efgh"
}
"""
print(addtone_prompt)

假设这是两个可爱的女孩子正在网上交流。女孩们都喜欢的话题是动漫、科技、宠物、甜点，是二次元宅女。她们对编程、电子、自然科学、法律金融等专业有深入了解，她们都喜欢使用颜文字。
请用更加符合人物设定的语气风格改写这段对话，不改变原文意思，不改变专业词汇。需要使用缩略语，网络用语，简称代称，省略主语定语等方法达成简洁的网络对话效果。可以加入语气词和从句，使用同音不同字的语气后缀。：

A：python sqlite3 中的fetchmany怎么用？
B：在Python的`sqlite3`模块中，`fetchmany`方法用于从查询结果中分批获取指定数量的记录，避免一次性加载所有数据到内存中，从而提高性能。通过指定参数，可以控制每次获取的记录数量，并且可以多次调用该方法以逐步获取所有记录。

按照json格式输出改写后的对话。示例输出：
{
    "A":"abcd",
    "B":"efgh"
}



In [ ]:
toneresp, _ = chat(client, addtone_prompt, verbose=False, temperature=0.7)
print(toneresp)

```json
{
    "A": "py里sqlite3的fetchmany咋用？",
    "B": "在py的`sqlite3`模块，`fetchmany`方法超实用，可以从查询结果里分批捞数据，避免一次全加载到内存，性能upup！指定参数就能控制每次捞的数量，多次调用慢慢捞完所有记录～"
}
```


In [ ]:
l = toneresp.find('```json')+7
r = toneresp.find('```',l)
print(f"Q: {quest}")
print(f"A: {json.loads(toneresp[l:r])["B"]}")

Q: python sqlite3 中的fetchmany怎么用？
A: 在py的`sqlite3`模块，`fetchmany`方法超实用，可以从查询结果里分批捞数据，避免一次全加载到内存，性能upup！指定参数就能控制每次捞的数量，多次调用慢慢捞完所有记录～
